In [ ]:
# Optional config for better memory efficiency
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
#import k3d

# Required imports
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

In [ ]:
from mapanything.models import MapAnything
from mapanything.utils.image import load_images

# Init model - This requires internet access or the huggingface hub cache to be pre-downloaded
# For Apache 2.0 license model, use "facebook/map-anything-apache"
model = MapAnything.from_pretrained("facebook/map-anything").to(device)

In [ ]:
# Load and preprocess images from a folder or list of paths
images = "/home/jovyan/work/Nextcloud/Nextcloudprintart/IST/Escolaridade/PIV2025/images"  # or ["path/to/img1.jpg", "path/to/img2.jpg", ...]
views = load_images(images)

In [ ]:
# Run inference
predictions = model.infer(
    views,                            # Input views
    memory_efficient_inference=False, # Trades off speed for more views (up to 2000 views on 140 GB)
    use_amp=True,                     # Use mixed precision inference (recommended)
    amp_dtype="bf16",                 # bf16 inference (recommended; falls back to fp16 if bf16 not supported)
    apply_mask=True,                  # Apply masking to dense geometry outputs
    mask_edges=True,                  # Remove edge artifacts by using normals and depth
    apply_confidence_mask=False,      # Filter low-confidence regions
    confidence_percentile=10,         # Remove bottom 10 percentile confidence pixels
)

# Access results for each view - Complete list of metric outputs
for i, pred in enumerate(predictions):
    # Geometry outputs
    pts3d = pred["pts3d"]                     # 3D points in world coordinates (B, H, W, 3)
    pts3d_cam = pred["pts3d_cam"]             # 3D points in camera coordinates (B, H, W, 3)
    depth_z = pred["depth_z"]                 # Z-depth in camera frame (B, H, W, 1)
    depth_along_ray = pred["depth_along_ray"] # Depth along ray in camera frame (B, H, W, 1)

    # Camera outputs
    ray_directions = pred["ray_directions"]   # Ray directions in camera frame (B, H, W, 3)
    intrinsics = pred["intrinsics"]           # Recovered pinhole camera intrinsics (B, 3, 3)
    camera_poses = pred["camera_poses"]       # OpenCV (+X - Right, +Y - Down, +Z - Forward) cam2world poses in world frame (B, 4, 4)
    cam_trans = pred["cam_trans"]             # OpenCV (+X - Right, +Y - Down, +Z - Forward) cam2world translation in world frame (B, 3)
    cam_quats = pred["cam_quats"]             # OpenCV (+X - Right, +Y - Down, +Z - Forward) cam2world quaternion in world frame (B, 4)

    # Quality and masking
    confidence = pred["conf"]                 # Per-pixel confidence scores (B, H, W)
    mask = pred["mask"]                       # Combined validity mask (B, H, W, 1)
    non_ambiguous_mask = pred["non_ambiguous_mask"]                # Non-ambiguous regions (B, H, W)
    non_ambiguous_mask_logits = pred["non_ambiguous_mask_logits"]  # Mask logits (B, H, W)

    # Scaling
    metric_scaling_factor = pred["metric_scaling_factor"]  # Applied metric scaling (B,)

    # Original input
    img_no_norm = pred["img_no_norm"]         # Denormalized input images for visualization (B, H, W, 3)

In [ ]:
confidence = pred["conf"]                 # Per-pixel confidence scores (B, H, W)
mask = pred["mask"]                       # Combined validity mask (B, H, W, 1)
non_ambiguous_mask = pred["non_ambiguous_mask"]                # Non-ambiguous regions (B, H, W)
non_ambiguous_mask_logits = pred["non_ambiguous_mask_logits"]  # Mask logits (B, H, W)


In [ ]:

from scipy.io import savemat
savemat("/Users/jpc/Downloads/PIV2025/images/taag.mat",{"many":predictions})


In [ ]:
import k3d
plot = k3d.plot()
plot += k3d.line([[0, 0, 0],
                 [1, 1, 1]])

plot.display()


In [ ]:
torch.cuda.is_available() 

In [ ]:
import mediapy as media
import numpy as np

url = 'https://github.com/hhoppe/data/raw/main/video.mp4'
video = media.read_video(url)
print(video.shape, video.dtype)  # It is a numpy array.
print(video.metadata.fps)  # The 'metadata' attribute includes framerate.
media.show_video(video)  # Play the video using the retrieved framerate.

media.show_images(video, height=80, columns=4)  # Show frames side-by-side.

video = media.moving_circle((128, 128), num_images=10)
media.show_video(video, fps=10)